<a href="https://colab.research.google.com/github/kgautam2103/Eventbrite_automation/blob/main/update_recurring_meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, timezone
from dateutil import parser
from google.colab import files

auth_token = ''
prod_url = 'https://www.eventbriteapi.com/v3/'
events= '/events/'
series = '/series/'
counter=1


current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
print(current_ts)
output_sheet = current_ts+'_recurring_update_output'
output_csv = current_ts+'_recurring_update_output.csv'

sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['parent_event_id','child_event_id','new_start_date','new_end_date','update_status','response_text'],counter)

#final_ouput_df = pd.DataFrame(columns=['parent_event_id','child_event_id','new_start_date','new_end_date','update_status','response_text' ])
final_output_list=[]

authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xADyxqRvw2YiCxSOGfCDcIR6MN9EY6klOSCC5XCruZE/edit#gid=0')

input_sheet = wb.worksheet('inputdata')
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['event_id','filter_start','filter_end','offset'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

for iter,row in inputdata_df.iterrows():
  event_id = row['event_id'].strip()
  filter_start = row['filter_start'].strip()
  #print(filter_start)
  filter_end = row['filter_end'].strip()
  #print(filter_end)
  offset = row['offset'].strip()

  child_event_id_list = []
  start_list = []
  end_list = []
  timezone_list =[]
  child_event_detail_df = pd.DataFrame(None,columns=['child_event_id','start','end','timezone'])

  if filter_start == "":
    #filter_start_date = parser.parse('2020-01-01T00:00:00Z')
    dt_now = datetime.now()
    filter_start_date = dt_now.replace(tzinfo=timezone.utc)
  else:
    filter_start_date = parser.parse(filter_start)


  if filter_end == "":
    filter_end_date = parser.parse('2030-01-01T00:00:00Z')
  else:
    filter_end_date = parser.parse(filter_end)

  print(filter_start_date)
  print(filter_end_date)
  list_event_series_url = prod_url+series+event_id+events

  print(list_event_series_url)
  list_event_series_response = requests.get(list_event_series_url,headers=auth_headers)

  event_details = []
  event_details.extend(list_event_series_response.json()['events'])

  has_more_items = list_event_series_response.json()['pagination']['has_more_items']

  while(has_more_items):
    continuation_code= list_event_series_response.json()['pagination']['continuation']
    list_event_series_url = prod_url+series+event_id+events+"/?continuation="+continuation_code
    list_event_series_response = requests.get(list_event_series_url,headers=auth_headers)
    event_details.extend(list_event_series_response.json()['events'])
    has_more_items = list_event_series_response.json()['pagination']['has_more_items']

  print('event_id is' +event_id)

  for event_detail in event_details:
    child_event_id_list.append(event_detail['id'])
    start_list.append(parser.parse(event_detail['start']['utc']))
    end_list.append(parser.parse(event_detail['end']['utc']))
    timezone_list.append(event_detail['start']['timezone'])

  child_event_detail_df['child_event_id'] = child_event_id_list
  child_event_detail_df['start'] = start_list
  child_event_detail_df['end'] = end_list
  child_event_detail_df['timezone'] = timezone_list

  child_event_detail_df = child_event_detail_df[ child_event_detail_df['start'] > filter_start_date]
  child_event_detail_df = child_event_detail_df[ child_event_detail_df['end'] < filter_end_date]

  for iter, row in child_event_detail_df.iterrows():
    counter = counter+1
    start_datetime = row['start'].to_pydatetime()
    end_datetime = row['end'].to_pydatetime()
    child_event_id = row['child_event_id']
    timezone_value = row['timezone']
    start_time_new = (start_datetime+timedelta(hours=float(offset))).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time_new = (end_datetime+timedelta(hours=float(offset))).strftime('%Y-%m-%dT%H:%M:%SZ')
    print('child_event_id is ' +child_event_id)
    print(start_time_new)
    update_request_body = {"event": {
        "start": {"timezone": timezone_value,"utc": start_time_new},
        "end": { "timezone": timezone_value,"utc": end_time_new}
    }
    }

    update_event_values = json.dumps(update_request_body)
    print(update_event_values)
    update_event_url = prod_url+events+child_event_id+"/"
    update_response = requests.post(update_event_url, data=update_event_values, headers=auth_headers)
    update_status = update_response.status_code
    update_response_text = update_response.text
    print(update_response_text)
    if update_status == 200:
      print(child_event_id+" is updated start and end datetime")
    output_value_list=[event_id,child_event_id,start_time_new,end_time_new,update_status,update_response_text]
    output_sheet.insert_row(output_value_list,counter)
    final_output_list.append(output_value_list)

#final_output_df = pd.DataFrame(final_output_list,columns=['parent_event_id','child_event_id','new_start_date','new_end_date','update_status','response_text' ])
#final_output_df.to_csv(output_csv)
#files.download(output_csv)
